This notebook explains how the `Scorer` class can be used to compute the following metrics: 
* MAE (mean absolute error); 
* MSE (mean squared error); 
* CRPS (Continuous Ranked Probability Score);
* Log Score.

The CRPS and Log score assume a normal distribution since the lower and upper 95% CI intervals are used to compute the score. 

In [1]:
import numpy as np
import pandas as pd
from mosqlient import get_infodengue
from mosqlient.models.score import Scorer

To show the use of the class to compare the score of new predictions with the predictions registered on the platform it will be used the baseline model.

In [2]:
from datetime import date
from mosqlient.datastore import Infodengue
from mosqlient.models.baseline import Arima

It will compare the predictions of id 77 and 78, which refer to the geocode 3304557, between `2022-01-02` and `2023-06-25`. 

So initially, we will get the data that will be used to train the model and generate the new predictions:

In [3]:
disease = 'dengue'
geocode = 3304557 
end_date = date.today().strftime('%Y-%m-%d')

df = get_infodengue(disease = 'dengue',
                start_date = '2010-01-01', 
              end_date = '2023-12-31', 
              geocode = 3304557)

df['data_iniSE'] = pd.to_datetime(df['data_iniSE'])

df.set_index('data_iniSE', inplace = True )
    
df = df[['casos']].rename(columns = {'casos':'y'})

df = df.resample('W-SUN').sum()

df.head()

,y
data_iniSE,
2010-01-03,30
2010-01-10,44
2010-01-17,46
2010-01-24,47
2010-01-31,68


Create the model:

In [4]:
m_arima = Arima(df = df)

m_arima

Train the model: 

In [5]:
model = m_arima.train(train_ini_date='2010-01-01', train_end_date = '2021-12-31')


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-292.056, Time=0.12 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-263.166, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-297.770, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-289.846, Time=0.04 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-265.069, Time=0.02 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-302.929, Time=0.04 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-302.144, Time=0.05 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-309.026, Time=0.16 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-301.338, Time=0.06 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-307.027, Time=0.14 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-305.684, Time=0.15 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-306.429, Time=0.25 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=-310.932, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-303.191, Time=0.04 sec
 ARIMA(2,1,0)(0,0,0)[0]          

Generate the out-of-sample predictions:

In [6]:
df_out = m_arima.predict_out_of_sample(horizon = 4, end_date = '2023-06-25', plot = False)

df_out.head()

,dates,preds,lower,upper,data
0,2022-01-02,3.614659,2.380141,5.611550,21.0
1,2022-01-09,3.433225,2.039727,5.982192,14.0
2,2022-01-16,3.252120,1.717641,6.491387,22.0
3,2022-01-23,3.118684,1.490893,7.007502,19.0
0,2022-01-30,21.331335,12.570924,37.513239,31.0


To compare the model with other predictions in the platform, it is necessary to provide the ids of the predictions, the dataframe of the new prediction with the columns `dates`, `preds`, `lower` and `upper`, and the dataframe with the `true` values to be compared (df_true). This dataframe must contain the columns `dates` and `casos`.

Redefine the data to compare the predictions:

In [7]:
data = df.reset_index()

data = data.rename(columns = {'data_iniSE': 'dates', 'y':'casos'})

data.head()

,dates,casos
0,2010-01-03,30.0
1,2010-01-10,44.0
2,2010-01-17,46.0
3,2010-01-24,47.0
4,2010-01-31,68.0


In [8]:
%%time
score = Scorer(df_true = data, ids = [77,78], preds = df_out)

CPU times: user 40.7 ms, sys: 6.05 ms, total: 46.8 ms
Wall time: 2.16 s


**The class above can be initialized with just the `ids` or `preds` parameter filled.** 

To see the MAE error for your model (key = `preds`) and the other use: 

In [9]:
score.mae

{'preds': 115.52500842999136,
 '77': 150.21846153846153,
 '78': 260.9831835557859}

To plot a bar chart: 

In [10]:
score.plot_mae()

alt.Chart(...)

To see the MSE error for your model (key = `preds`) and the other use: 

In [11]:
score.mse

{'preds': 39545.323650490725, '77': 69988.36318717948, '78': 166280.6349972801}

To plot a bar chart: 

In [12]:
score.plot_mse()

alt.Chart(...)

To see the CRPS score for your model (key = `preds`) and the other, use the code below. The first dict contains the score by each point, and the second one shows the mean of the score.  

In [13]:
score.crps

({'preds': dates
  2022-01-02     16.929559
  2022-01-09     10.010700
  2022-01-16     18.074555
  2022-01-23     15.103212
  2022-01-30      6.475991
                   ...    
  2023-05-28    392.053336
  2023-06-04    549.736236
  2023-06-11    704.717576
  2023-06-18    159.772491
  2023-06-25    261.630103
  Length: 78, dtype: float64,
  '77': dates
  2022-01-02    102.524694
  2022-01-09     89.908531
  2022-01-16     39.430201
  2022-01-23     67.323246
  2022-01-30     61.630854
                   ...    
  2023-05-28    129.618554
  2023-06-04    749.611256
  2023-06-11    601.061591
  2023-06-18    149.510736
  2023-06-25    222.354416
  Length: 78, dtype: float64,
  '78': dates
  2022-01-02     59.172107
  2022-01-09     31.326248
  2022-01-16     25.620518
  2022-01-23     27.721827
  2022-01-30     38.027232
                   ...    
  2023-05-28    365.740799
  2023-06-04    187.866280
  2023-06-11    469.313500
  2023-06-18    312.681174
  2023-06-25    422.268150
  Le

To plot the score you can use: 

In [14]:
score.plot_crps()

alt.VConcatChart(...)

To see the Log score for your model (key = `preds`) and the other, use the code below. The first dict contains the score by each point, and the second one shows the mean of the score.  

In [15]:
score.log_score

({'preds': dates
  2022-01-02   -232.270353
  2022-01-09    -58.374197
  2022-01-16   -124.484547
  2022-01-23    -67.541155
  2022-01-30     -3.951334
                   ...    
  2023-05-28     -8.223276
  2023-06-04     -8.592945
  2023-06-11     -8.895450
  2023-06-18     -7.435560
  2023-06-25     -7.828248
  Length: 78, dtype: float64,
  '77': dates
  2022-01-02   -6.992415
  2022-01-09   -6.870076
  2022-01-16   -6.046198
  2022-01-23   -6.580273
  2022-01-30   -6.493813
                  ...   
  2023-05-28   -7.019283
  2023-06-04   -9.422886
  2023-06-11   -8.505237
  2023-06-18   -7.281535
  2023-06-25   -7.342796
  Length: 78, dtype: float64,
  '78': dates
  2022-01-02   -6.236466
  2022-01-09   -5.808406
  2022-01-16   -5.562729
  2022-01-23   -5.662115
  2022-01-30   -5.963385
                  ...   
  2023-05-28   -7.901946
  2023-06-04   -7.206261
  2023-06-11   -8.591845
  2023-06-18   -7.677499
  2023-06-25   -8.896633
  Length: 78, dtype: float64},
 {'preds': -12.11

To plot this score: 

In [16]:
score.plot_log_score()

alt.VConcatChart(...)

To generate a table with a summary of the scores, you can use the following: 

In [17]:
score.summary

,mae,mse,crps,log_score
id,,,,
preds,115.525008,39545.323650,121.892209,-12.117806
77,150.218462,69988.363187,156.069647,-7.138349
78,260.983184,166280.634997,194.826199,-7.206595


To generate a plot of the predictions, you can use the following: 

In [18]:
score.plot_predictions()

alt.LayerChart(...)

The class will select the bigger range of dates that contains information about each prediction dataframe. If you want to see the model's performance in a lower range, use the method below. The new range of dates provided must be between `score.min_date` and `score.max_date`. Otherwise, it will return an error.

In [19]:
score.set_date_range(start_date = '2023-01-01', end_date='2023-06-01' )

In this case:

In [20]:
score.summary

,mae,mse,crps,log_score
id,,,,
preds,224.814969,90991.537676,264.947337,-7.430783
77,204.412273,76788.430632,179.290006,-7.322140
78,588.199306,506643.235469,457.150661,-9.203940


In [21]:
score.plot_log_score()

alt.VConcatChart(...)

In [22]:
score.plot_predictions()

alt.LayerChart(...)